In [ ]:
# default_exp forecast

# forecast

> Módulo com implementação de algoritmos para detecção de anomalias em series temporais utilizando métodos de **forecast**:

* **ARIMA**
* **Holt-Winters**

In [ ]:
#export
import numpy as np
import pandas as pd

import warnings
import pmdarima as pm

## ARIMA

In [ ]:
#export
def auto_arima(ts, max_p=5, max_q=5, trace=True):
    ''' returns ARIMA model for **ts** '''
    #ignore  statmodels warning
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        model = pm.auto_arima(df['value'].values, max_p=max_p, max_q=max_q)

    return model

In [ ]:
#export
def check_interval(ts_orig, conf_inter):
    ''' retornar os pontos da serie t'emporal **ts_orig** 
        que estão dentro do intervalo de confiança
        
        Inputs:
        ts_orig: pd.Series()
        conf_inter: np.array([[0, 1], [1, 2], ..., [1, 2]]) 
                    shape: (N_examples, 2)
        
        returns:
        Series contendo os pontos dentro do intervalo de confiança 
        '''
    df_conf = pd.DataFrame(conf_inter)
    anomalies = ((ts.values >= tmp[0].values)) & ((ts.values < tmp[1].values))
    return ts_orig[~anomalies]

In [ ]:
#export
def arima_anomalies(ts, model):
    '''
    returns anomalies based on arima forecast
    ts: Series
    model: ARIMAmodel from pdmarima
    '''
    conf_inter = model.predict_in_sample(return_conf_int=True)[1]
    return check_interval(ts, conf_inter)